## Prototype for Conformalized Survival Analysis with Right-Censoring

This notebook demonstrates the performance of a new prototype method to calibrate lower confidence bounds for survival times under right-censoring.
This method used imputation of unobserved censoring times to obtain an artificial data set with type-I censoring, and then applies Candes' method.
The methods are implemented in the accompanying "utils" R scripts.

In [ ]:
# Load required libraries
suppressMessages(library(tidyverse))
suppressMessages(library(gridExtra))
library(survival)
library(ggsurvfit)
library(grf)
library(cfsurvival)

## Generate some survival data

In [ ]:
# Generate data from the Log-normal model (Mvt + Heterosc.) of Candes et al. (Table 1)
source("../conf_surv/utils_data_new.R")

setting = 7

if(setting==1) {
    ## Setting 1 from Gui et al.
    ## Initialize the covariate model
    num_features <- 1
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 4), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + 0.632 * X[,1]
    surv_sigma_fun <- function(X) 0*X[,1] + 2
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + 0.1
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)

} else if(setting==2) {
    ## Setting 2 from Gui et al.
    ## Initialize the covariate model
    num_features <- 1
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 4), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + 3 * (X[,1]>2) + X[,1] * (X[,1]<2)
    surv_sigma_fun <- function(X) 0*X[,1] + 0.5
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + 0.1
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)

} else if(setting==3) {
    ## Setting 3 from Gui et al.
    ## Initialize the covariate model
    num_features <- 1
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 4), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + 2 * (X[,1]>2) + X[,1] * (X[,1]<2)
    surv_sigma_fun <- function(X) 0*X[,1] + 0.5
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + 0.25 + (X[,1]+6)/100
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)

    
} else if(setting==4) {
    ## Setting 4 from Gui et al.
    ## Initialize the covariate model
    num_features <- 1
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 4), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + 3 * (X[,1]>2) + 1.5 * X[,1] * (X[,1]<2)
    surv_sigma_fun <- function(X) 0*X[,1] + 0.5
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_mu_fun <- function(X) 0*X[,1] + 2 + (2-X[,1])/50
    cens_sigma_fun <- function(X) 0*X[,1] + 0.5
    censoring_generator <- LogNormalDistribution$new(mu_fun = cens_mu_fun, sigma_fun = cens_sigma_fun)
    
} else if(setting==5) {
    ## Setting 5 from Gui et al.
    ## Initialize the covariate model
    num_features <- 10
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 4), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + 0.126 * (X[,1] + sqrt(X[,3]*X[,5])) + 1
    surv_sigma_fun <- function(X) 0*X[,1] + 0.5
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + X[,10]/10 + 1/20
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)

} else if(setting==6) {
    ## Setting 6 from Gui et al.
    ## Initialize the covariate model
    num_features <- 10
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 4), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + 0.126 * (X[,1] + sqrt(X[,3]*X[,5])) + 1
    surv_sigma_fun <- function(X) 0*X[,1] + (X[,2]+2)/4
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + X[,10]/10 + 1/20
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)
    
} else if(setting==7) {
    ## New setting
    ## Initialize the covariate model
    num_features <- 100
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 1), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + X[,1]^0.25
    surv_sigma_fun <- function(X) 0*X[,1] + 0.1
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_mu_fun <- function(X) 0*X[,1] + X[,1]^4 + 0.4
    cens_sigma_fun <- function(X) 0*X[,1] + 0.1
    censoring_generator <- LogNormalDistribution$new(mu_fun = cens_mu_fun, sigma_fun = cens_sigma_fun)

} else if(setting==8) {
    ## New setting
    ## Initialize the covariate model
    num_features <- 20
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, 0, 1), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + (X[,2]>0.5) + (X[,3]<0.5) + (1-X[,1])^0.25
    surv_sigma_fun <- function(X) 0*X[,1] + 0.1 * (1-X[,1])
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_mu_fun <- function(X) 0*X[,1] + (X[,2]>0.5) + (X[,3]<0.5) + (1-X[,1])^4 + 0.4
    cens_sigma_fun <- function(X) 0*X[,1] + 0.1 * X[,2]
    censoring_generator <- LogNormalDistribution$new(mu_fun = cens_mu_fun, sigma_fun = cens_sigma_fun)

} else if(setting==9) {
    ## Setting 3 from Candes et al.
    ## Initialize the covariate model
    num_features <- 100
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, -1, 1), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + log(2) + 1 + 0.55*(X[,1]^2-X[,3]*X[,5])
    surv_sigma_fun <- function(X) 0*X[,1] + 1
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + 0.4
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)

} else if(setting==10) {
    ## Setting 4 from Candes et al.
    ## Initialize the covariate model
    num_features <- 100
    covariate_generator <- function(num_samples) {
        matrix(runif(num_samples * num_features, -1, 1), nrow = num_samples)
    }
    ## Initialize the survival time distribution
    surv_mu_fun <- function(X) 0*X[,1] + log(2) + 1 + 0.55*(X[,1]^2-X[,3]*X[,5])
    surv_sigma_fun <- function(X) 0*X[,1] + abs(X[,10]) + 1
    survival_generator <- LogNormalDistribution$new(mu_fun = surv_mu_fun, sigma_fun = surv_sigma_fun)
    ## Initialize the censoring time distribution
    cens_rate_fun <- function(X) 0*X[,1] + 0.4
    censoring_generator <- ExponentialDistribution$new(rate_fun = cens_rate_fun)
}

# Initialize the data generator
generator <- SurvivalDataGenerator$new(covariate_generator, survival_generator, censoring_generator)

    
# Sample sizes
num_samples_train <- 100
num_samples_cal <- 100
num_samples_test <- 1000
num_feat_censor <- 100

# Set random seet
#set.seed(2024)

# Generate training, calibration, and test data
# (including true event and censoring times)
data.train.oracle <- generator$sample(num_samples_train)
data.cal.oracle <- generator$sample(num_samples_cal)
data.test.oracle <- generator$sample(num_samples_test)

# Remove true event and censoring times from the data (right-censoring)
data.train <- data.train.oracle |> select(-event_time, -censoring_time)
data.cal <- data.cal.oracle |> select(-event_time, -censoring_time)
data.test <- data.test.oracle |> select(-event_time, -censoring_time)

# Remove only true event times from the data (type-I censoring)
data.train.t1 <- data.train.oracle |> select(-event_time)
data.cal.t1 <- data.cal.oracle |> select(-event_time)
data.test.t1 <- data.test.oracle |> select(-event_time)

#generator$predict_quantiles(select(data.test, -time, -status)[c(24),], T.max=1000)

## Visualize the true (oracle) data distribution

In [ ]:
data.test.oracle |>
    select(X1, event_time, censoring_time) |>
    pivot_longer(cols = c(event_time, censoring_time), 
                 names_to = "time_type", values_to = "time_value") |>
    ggplot(aes(x=X1, y=time_value, color=time_type)) +
    geom_point() +
    scale_y_continuous(trans='log10') +
    theme_bw()

In [ ]:
source("../conf_surv/utils_conformal.R")

alpha = 0.1

pred.oracle.fast <- generator$survival$predict_quantiles(select(data.test, -time, -status), probs=c(alpha))[[1]]

evaluate_bounds(data.test$time, 
                pred.oracle.fast,
                event_time=data.test.oracle$event_time)

In [ ]:
# Plot the survival curves

source("../conf_surv/utils_plotting_new.R")
predictions.surv = generator$survival$predict(data.test[1:4,], seq(0,5, length.out=1000))
plot_survival_curves(predictions.surv)

In [ ]:
# Plot the censoring curves

source("../conf_surv/utils_plotting_new.R")
predictions.censor = generator$censoring$predict(data.test[1:4,], seq(0,20, length.out=1000))
plot_survival_curves(predictions.censor)

## Explore the data

Let's take a look at the data

In [ ]:
# Summarize data under type-I censoring
data.train.oracle |>
    summarise(Observations=n(), Events = sum(status), 
              `Proportion censored`=mean(status==0),
              `Median event time`=median(event_time),
              `Mean event time`=mean(event_time),
              `Median censoring time`=median(censoring_time),
              `Mean censoring time`=mean(censoring_time),
              `Median observed time`=median(time),
              `Median observed event time`=median(time[status]),
              `Median censoring time` = median(censoring_time))

In [ ]:
# Kaplan-Meier plot
survfit2(Surv(time, status) ~ 1, data = data.train) |> 
  ggsurvfit() +
  labs(
    x = "Time",
    y = "Overall survival probability"
  ) +
  add_confidence_interval() +
  add_risktable()

## Fit survival model

Fit a survival model using the training data.

In [ ]:
source("../conf_surv/utils_survival_new.R")

model_type = "grf"

if(model_type=="grf") {
    # Instantiate GRF survival forest
    surv_model <- GRF_SurvivalForestWrapper$new()
} else if (model_type=="survreg") {
    # Instantiate Cox proportional hazards model
    surv_model <- SurvregModelWrapper$new(dist="lognormal")
} else if (model_type=="rf") {
    # Instantiate random forest survival model
    surv_model <- randomForestSRC_SurvivalWrapper$new()
} else if (model_type=="cox") {
    # Instantiate random forest survival model
    surv_model <- CoxphModelWrapper$new()
} else {
    stop("Unknown model type!")
}

## Visualize the model's predictions

Visualize the predicted survival curves for two individuals.

In [ ]:
surv_model$fit(Surv(time, status) ~ ., data = data.train)

idx.predict <- c(1:4)

predictions = surv_model$predict(data.cal[idx.predict,])
plot_survival_curves(predictions)

Predict survival time quantiles

In [ ]:
# Predict survival time quantiles
surv_model$predict_quantiles(data.cal[idx.predict,], probs = c(0.1, 0.5, 0.9))

## Estimating the censoring distribution

Let's start by visualizing the true censoring distribution

In [ ]:
source("../conf_surv/utils_censoring.R")
source("../conf_surv/utils_survival_new.R")

# List of covariates to use for censoring model
use.covariates <- paste("X",1:min(num_features,num_feat_censor),sep="")

# Initialize a base model for the censoring distribution
#cens_base_model <- GRF_SurvivalForestWrapper$new(use_covariates=use.covariates)
cens_base_model <- SurvregModelWrapper$new(dist="lognormal", use_covariates=use.covariates)
#cens_base_model <- CoxphModelWrapper$new(use_covariates=use.covariates)
#cens_base_model <- randomForestSRC_SurvivalWrapper$new(use_covariates=use.covariates)

# Create an instance of the CensoringModel class with the model
censoring_model <- CensoringModel$new(model = cens_base_model)
censoring_model <- CensoringModel$new(model = cens_base_model)

# Fit the censoring distribution
censoring_model$fit(data = data.train)

# Plot the censoring distribution for the first individual
idx.predict <- c(1:4)
pred_surv = censoring_model$model$predict(data.cal[idx.predict,])
plot_survival_curves(pred_surv)

## Oracle prediction bounds

In [ ]:
source("../conf_surv/utils_conformal.R")

alpha = 0.1

pred.oracle <- generator$survival$predict_quantiles(select(data.test, -time, -status), probs=c(alpha))[[1]]

evaluate_bounds(data.test$time, 
                pred.oracle,
                event_time=data.test.oracle$event_time,
                oracle = pred.oracle)

## Naive CQR

In [ ]:
source("../conf_surv/utils_conformal.R")

alpha = 0.1
pred.cqr <- predict_CQR(data.test, surv_model, data.cal, alpha)

evaluate_bounds(data.test$time, 
                pred.cqr,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Nominal lower bound

In [ ]:
alpha = 0.1
pred.nominal <- surv_model$predict_quantiles(data.test, probs=alpha)[[1]]

evaluate_bounds(data.test$time, 
                pred.nominal,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## CQR with de-censoring

In [ ]:
## Create a survival object
surv_object <- survival::Surv(time = data.train$time, event = data.train$status)

## Fit the Kaplan-Meier survival model
km_fit <- survfit(surv_object ~ 1)

## Plot the Kaplan-Meier survival curve
plot(km_fit)

In [ ]:
source("../conf_surv/utils_decensoring.R")

pred.decensor <- predict_decensoring(data.test, surv_model, km_fit, data.cal, alpha, R=10)

evaluate_bounds(data.test.oracle$time, 
                pred.decensor,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Gui's method (with oracle censoring times, and oracle weights)

In [ ]:
source("../conf_surv/utils_conformal.R")

# Impute the missing censoring times
C.cal <- data.cal.oracle$censoring_time

pred.gui <- predict_Gui(data.test, surv_model, generator$censoring, data.cal, C.cal, alpha, use_cqr=FALSE, use_censoring_model=TRUE, 
                        finite_sample_correction=FALSE)

evaluate_bounds(data.test.oracle$time, 
                pred.gui,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Candes' method (with oracle censoring times, and oracle weights)

In [ ]:
source("../conf_surv/utils_conformal.R")

# Impute the missing censoring times
C.cal <- data.cal.oracle$censoring_time

pred.candes <- predict_Candes(data.test, surv_model, generator$censoring, data.cal, C.cal, alpha)

evaluate_bounds(data.test.oracle$time, 
                pred.candes,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

In [ ]:
source("../conf_surv/utils_conformal.R")

# Apply Candes' method with automatic tuning of c0
tuning.package <- list(data.train = data.train,
                       C.train = data.train.oracle$censoring_time)

pred.candes.tuned <- predict_Candes(data.test, surv_model, generator$censoring, data.cal, C.cal, alpha, 
                                    tuning.package=tuning.package)

evaluate_bounds(data.test.oracle$time, 
                pred.candes.tuned,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Candes' method (with oracle censoring times, and estimated weights)

In [ ]:
source("../conf_surv/utils_conformal.R")

# Impute the missing censoring times
C.cal <- data.cal.oracle$censoring_time

pred.candes <- predict_Candes(data.test, surv_model, censoring_model$model, data.cal, C.cal, alpha)

evaluate_bounds(data.test.oracle$time, 
                pred.candes,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

In [ ]:
source("../conf_surv/utils_conformal.R")

# Tune c0
pred.candes.tuned <- predict_Candes(data.test, surv_model, censoring_model$model, data.cal, C.cal, alpha,
                                    tuning.package=tuning.package)

evaluate_bounds(data.test.oracle$time, 
                pred.candes.tuned,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Prototype (with Candes' method, imputation based on estimated censoring model, estimated weights)

In [ ]:
source("../conf_surv/utils_conformal.R")

tuning.package.2 <- list(data.train = data.train)

pred.proto <- predict_prototype(data.test, surv_model, censoring_model, data.cal, alpha, tuning.package=tuning.package.2, cutoffs="candes-tuning")

evaluate_bounds(data.test$time, 
                pred.proto,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Prototype (with Gui's method, imputation based on estimated censoring model, estimated weights)

In [ ]:
source("../conf_surv/utils_conformal.R")

pred.proto <- predict_prototype(data.test, surv_model, censoring_model, data.cal, alpha, cutoffs="adaptive-cqr", finite_sample_correction=FALSE)

evaluate_bounds(data.test$time, 
                pred.proto,
                event_time=data.test.oracle$event_time,
                oracle=pred.oracle)

## Prepare for repeated experiments

In [ ]:
source("../conf_surv/utils_conformal.R")
#surv_model <- GRF_SurvivalForestWrapper$new()
#cens_base_model <- SurvregModelWrapper$new(dist="weibull")

analyze_data <- function(data.train, data.cal, data.test, 
                         surv_model, cens_model, generator=NULL) {
    
    # Initialize the censoring model
    cens_model <- CensoringModel$new(model = cens_base_model)
    
    # Fit the survival model on the training data
    surv_model$fit(Surv(time, status) ~ ., data = data.train)
    
    # Fit the censoring model on the training data
    cens_model$fit(data = data.train)
    
    # Apply all methods
    predictions <- c()
    
    # Construct oracle lower bound
    if(!is.null(generator)) {
        predictions$oracle <- generator$survival$predict_quantiles(select(data.test, -time, -status), probs=c(alpha))[[1]]
    }
    
    # Construct nominal lower bound
    predictions$nominal <- surv_model$predict_quantiles(data.test, probs=alpha)[[1]]
    
    # Apply naive CQR
    predictions$cqr <- predict_CQR(data.test, surv_model, data.cal, alpha)

    # Apply CQR with de-censoring
    predictions$cqr.decensor <- predict_decensoring(data.test, surv_model, km_fit, data.cal, alpha, R=10)
    
    # Apply Candes' method with "oracle" censoring model
    C.cal.oracle <- data.cal.oracle$censoring_time
    predictions$candes.oracle <- predict_Candes(data.test, surv_model, generator$censoring, data.cal, C.cal.oracle, alpha)
    
    # Apply Candes' method with "oracle" censoring model and tuning
    #tuning.package <- list(data.train = data.train, C.train = data.train.oracle$censoring_time)
    #predictions$candes.oracle.tuned <- predict_Candes(data.test, surv_model, generator$censoring, data.cal, C.cal.oracle, alpha, tuning.package=tuning.package)
    
    # Apply Gui's method with "oracle" censoring model
    predictions$gui.oracle <- predict_Gui(data.test, surv_model, generator$censoring, data.cal, C.cal.oracle, alpha, finite_sample_correction=FALSE)
    
    # Apply prototype (Candes)
    predictions$prototype.candes <- predict_prototype(data.test, surv_model, cens_model, data.cal, alpha, cutoffs="candes-fixed")

    # Apply prototype (Gui)
    predictions$prototype.gui <- predict_prototype(data.test, surv_model, cens_model, data.cal, alpha, cutoffs="adaptive-cqr", 
                                                   finite_sample_correction=FALSE)

    
    return(predictions)
}

predictions <- analyze_data(data.train, data.cal, data.test, surv_model, censoring_model, generator=generator)

In [ ]:
results <- do.call(rbind, lapply(names(predictions), function(name) {
    res = evaluate_bounds(data.test$time, predictions[[name]], 
                          event_time=data.test.oracle$event_time,
                          oracle=predictions$oracle)
    
  cbind(Method = name, res)
}))
results

## TODO

 - Design more interesting data distribution
 - Run large-scale experiments